In [1]:
import pandas as pd
import numpy as np
import json
from dateutil import rrule, parser
from dataclasses import dataclass, field
from datetime import date, datetime, timedelta
from intervaltree import Interval, IntervalTree

# Definition of the different classes
@dataclass
class Operator:
    name: str
    skills: list
    holidays: IntervalTree
    availability: IntervalTree = field(default_factory=IntervalTree)
    shift: list = field(default_factory=list)   # Is it used ?

@dataclass
class Step:
    name: str
    previous_steps: list
    duration: timedelta
    required: timedelta
    capacity: int
    log: pd.DataFrame

    def __repr__(self):
        return f"Step {self.name}, duration {self.duration}, capacity {self.capacity}, log \n{self.log}"
    
# Load data from the JSON configuration file
with open("SimulatorInputs.json", "r") as file:
    data = json.load(file)

# Extract the simulation parameters
json_simulation_start: str = data['SimulationParameters']['simulation_start']
simulation_start: datetime = datetime.fromisoformat(json_simulation_start)

json_simulation_end: str = data['SimulationParameters']['simulation_end']
simulation_end: datetime = datetime.fromisoformat(json_simulation_end)

#modules_planned_deliveries: str = data["ComponentArrivalTimes"]["bare modules"]

# Initialize instances of the operator class
# All the operators are stored in the operators list
operators = []

# Initialize the CERN holidays to the good format
CERN_holidays_series = pd.Series(data["CERNHolidays"])
CERN_holidays_series[:] = CERN_holidays_series.apply(lambda x: [datetime.fromisoformat(date) for date in x])

for id, operator in enumerate(data["Operators"]):
    # Loads the operator's individual holidays
    individual_holidays = pd.Series(data["OperatorHolidays"][f"Operator{id + 1}"])
    individual_holidays[:] = individual_holidays.apply(lambda x: [datetime.fromisoformat(date) for date in x])

    # Merges the individual holidays with the CERN holidays
    operator_holidays = pd.concat([individual_holidays, CERN_holidays_series])
    operator_holidays[:] = operator_holidays.apply(lambda x: Interval(x[0], x[1], "holiday"))

    # Converts the pandas series to a list and then to an interval tree
    operator_holidays_list = operator_holidays.tolist()
    operator_holidays_tree = IntervalTree(operator_holidays_list)

    globals()[f"Operator{id + 1}"] = Operator(name=f"Operator{id + 1}", skills=data["Operators"][operator], holidays=operator_holidays_tree)
    operators.append(globals()[f"Operator{id + 1}"])


# Initialization of the Step class instances, they are stored in the "Chronologically_Ordered_Steps" dict, the member log is a dataframe that will contain 
# the entry and exit dates of the modules at each step

Chronologically_Ordered_Steps = {}
for step in data["StagesAndSteps"]:
    step_name = step["Step"].replace("-", "_").replace(" ", "_")    # Just converting the name to python valid name without blank spaces and


    if step["Previous"][0] == "None":
        globals()[step_name] = Step(name=step["Step"], previous_steps=[None], duration=timedelta(minutes = step["Duration"]),required=timedelta(minutes = step["Required"]), capacity= step["Capacity"],log=pd.DataFrame(columns=["Entry_Date", "Exit_Date"]))

    else:
        previous_steps = []
        for prev_step in step["Previous"]:
            previous_steps.append(Chronologically_Ordered_Steps[prev_step])
            globals()[step_name] = Step(name=step["Step"], previous_steps=previous_steps, duration=timedelta(minutes = step["Duration"]), required=timedelta(minutes = step["Required"]), capacity= step["Capacity"],log=pd.DataFrame(columns=["Entry_Date", "Exit_Date"]))
    Chronologically_Ordered_Steps |= {step["Step"]: globals()[step_name]}



finished_modules_count: int = sum(S___PDB_Shipment_of_modules_to_loading_sites.log["Entry_Date"].notna())

############################################################################################
def is_task_assignable(tree: IntervalTree, task_duration: Interval) -> bool:    #TODO: Manually implement IntervalTree.py and add this as a method of IntervalTree.
    assignable: bool = False
    for interval in tree:
        if interval.begin <= task_duration.begin and task_duration.end <= interval.end:
            assignable = True
            break
    return assignable
############################################################################################
############################################################################################
# Extract the holidays, and daily shift from the data
dict_int_to_day = {"0": "Monday", "1": "Tuesday", "2": "Wednesday", "3": "Thursday", "4": "Friday"}

def generate_operators_availability(date: datetime) -> None:
# Let's try to generate the availability of the operator day wise
    weekday: str = str(date.weekday())

    for operator in operators:
    # Reset the availability of the operator first
        operator.availability = IntervalTree()

        if len(operator.holidays[date]) or date.weekday() in {5,6}:  # Check if the day is either in the holidays or weekend
            continue

        else:

            today_shift = data["OperatorWorkHours"][operator.name][dict_int_to_day[weekday]]
            list_today_shift = list(today_shift.keys())

            # TODO: check for even number of time constraints
            beginning_slot = list_today_shift[::2]  # Extract the beginning and ending time of the different operator's daily slots
            ending_slot = list_today_shift[1::2]    # Implicitly we assume that there is an even number of time constraints (namely the beginning and the ending of the time slot)

            for begin, end in zip(beginning_slot, ending_slot):
                start_time = str(date)[:10] + " " + today_shift[begin]
                start_time = datetime.fromisoformat(start_time)
                end_time = str(date)[:10] + " " + today_shift[end]
                end_time = datetime.fromisoformat(end_time)
                operator.availability.add(Interval(start_time, end_time, "idle"))
############################################################################################
############################################################################################
def generate_lab_hours(date: datetime) -> IntervalTree:
    lab_hours: IntervalTree = IntervalTree()
    generate_operators_availability(date)
    for operator in operators:
        lab_hours |= operator.availability
    return lab_hours
############################################################################################
############################################################################################
# Load the state of production csv data into each step
state_of_production = pd.read_csv("inventory.csv", sep=";", dtype={"Quantity":'Int64'}, index_col=0, parse_dates=["Launching Time"])
state_of_production.fillna({"Quantity": 0}, inplace=True)

total_modules_number: int = int(sum(state_of_production.loc[:,"Quantity"]))
#for delivery in modules_planned_deliveries:
#    total_modules_number += modules_planned_deliveries[delivery]

# This piece of code fills by default the missing values of the Ready components launching time
# to simulation_start_time - duration of the task to simulate the fact that they are just ready to 
# be moved to the next step when the simulation is launched. 
for index, row in state_of_production.iterrows():   # I prefer to work with literal values of Index and Columns rather than the integer values for more reliability and legibility
    if row["Quantity"] > 0:     # We only care about the Steps where there are modules
        for task in iter(Chronologically_Ordered_Steps):
            if task in index:   # Looks for the task associated with the row TODO : suppress this loop by implementing yet another lookup table
                if pd.isna(state_of_production.loc[index, "Launching Time"]):
                    Time_ready_by_simulation_start = simulation_start - Chronologically_Ordered_Steps[task].duration  # In the last part we extract the duration associated with the task
                    state_of_production.loc[index, "Launching Time"] = Time_ready_by_simulation_start

# Now we fill all those initial data into the log attribute of each step
for Step in Chronologically_Ordered_Steps.values():
    row_Ready = state_of_production.loc[Step.name + " Ready"]
    row_WIP = state_of_production.loc[Step.name + " WIP"]
    df_Ready = pd.DataFrame([[row_Ready.loc["Launching Time"],pd.NaT] for _ in range(row_Ready.loc["Quantity"])],columns=["Entry_Date", "Exit_Date"])
    df_WIP = pd.DataFrame([[row_WIP.loc["Launching Time"],pd.NaT] for _ in range(row_WIP.loc["Quantity"])],columns=["Entry_Date", "Exit_Date"])
    Step.log = pd.concat([Step.log, df_Ready, df_WIP], ignore_index=True)   # TODO : maybe numpy arrays are better suited in this context, since it's a single type of datatype, in addition to that this line raises a warning

def tasks_by_priority(time: timedelta) -> list: # Returns the list of tasks that can be done at the current time

    steps_to_do = []

    for step in list(reversed(Chronologically_Ordered_Steps.values())): 

        # First step, check if the step is ready to process new modules
        time_array: np.ndarray = np.full_like(step.log.loc[:,"Entry_Date"], time)
        time_spent_in_task: pd.DataFrame = time_array - step.log.loc[:,"Entry_Date"]

        # We check if the number of modules being processed at the moment is greater than the capacity of the step

        condition = sum((time_spent_in_task < step.duration) & (pd.isna(step.log["Exit_Date"]))) >= step.capacity 

        if condition:  # If the step is not ready to process new modules
            continue    # we break the loop and go to the next step

        if step.previous_steps[0] is None:  # If previous_steps is None. That means that this the first step                        
            continue    # By default the first step is always ready, in fact that means that there are not enough components left

        else:
            # Second step, we compute the reception capacity of the step
            reception_capacity = step.capacity - sum(pd.isna(step.log["Exit_Date"]))  # We compute the number of modules ready to be processed in the next step
            
            # Now we check if among the previous steps there are enough modules ready to be processed in the next step
            ready_to_be_processed: bool = True
            modules_ready_overall: int = np.inf
            for previous_step in step.previous_steps:   

                previous_time_array: np.ndarray = np.full_like(previous_step.log.loc[:,"Entry_Date"], time) 
                time_spent_in_previous_task: pd.DataFrame = previous_time_array - previous_step.log.loc[:,"Entry_Date"]   # Displays the time spent in the previous task for each module
                modules_ready: int = sum((time_spent_in_previous_task >= previous_step.duration) & (pd.isna(previous_step.log["Exit_Date"])))      # If there are not enough modules ready to be processed in the previous steps
                modules_ready_overall = min(modules_ready, modules_ready_overall)  # We take the minimum of the modules ready in the previous steps
                if modules_ready <= 0:                                                       
                    ready_to_be_processed = False
            
            if ready_to_be_processed:
                for _ in range(min(reception_capacity, modules_ready_overall)): #TODO: list comprehension to fasten the loop
                    steps_to_do.append(step)

    return steps_to_do
############################################################################################
############################################################################################
def is_task_assignable(tree: IntervalTree, task_duration: Interval) -> bool:    #TODO: Manually implement IntervalTree.py and add this as a method of IntervalTree.
    assignable: bool = False
    for interval in tree:
        if interval.begin <= task_duration.begin and task_duration.end <= interval.end:
            assignable = True
            break
    return assignable
############################################################################################
############################################################################################
def get_next_available_time_for_task(time: datetime, task: Step) -> datetime:
    task_duration: Interval = Interval(time, time + task.required, task.name)
    operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

    new_time: datetime = time
    while len(operators_available) < 2:
        lab_hours = generate_lab_hours(new_time)
        has_moved: bool = False
        for interval in sorted(lab_hours):
            if interval.begin > time:
                time = interval.begin
                has_moved = True
                break

        if has_moved == False:
            new_time += timedelta(days=1)
        else:
            task_duration: Interval = Interval(time, time + task.required, task.name)
            operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

    return time
############################################################################################
############################################################################################
# Once a task is assigned to operators, a module has to be withdrawn 
# from the previous step and added to the next step
def update_log(task, time):

    # First step, we remove the module from the previous step
    if task.previous_steps[0] is None:  # There is no need to update the log for the first step
        pass
    else:
        for previous_step in task.previous_steps:
            condition = (previous_step.log["Entry_Date"] + previous_step.duration <= time) & (pd.isna(previous_step.log["Exit_Date"]))    # Filter all the modules ready that have not been moved yet
                # Check if any rows match the condition
            if condition.any():
                # Get the first index that matches the condition
                first_matching_index = previous_step.log[condition].index[0]    # Take the first ready module and fill its exit date
                
                # Update the 'Exit_Date' for the first matching row
                previous_step.log.at[first_matching_index, "Exit_Date"] = time
            else:
                print("There is a big issue !!")

    # Second step, we add the module to the next step
    df = pd.DataFrame([[time, pd.NaT]], columns=["Entry_Date", "Exit_Date"])
    task.log = pd.concat([task.log, df], ignore_index=True)
############################################################################################
############################################################################################
# What is that, where does this come from? I haven't coded that

"""def get_next_available_time(time: datetime) -> datetime:
    # Get the next working time slot
    next_available_time: datetime = time
    for operator in operators:
        if operator.availability.overlaps(task_duration):
            next_available_time = max(next_available_time, operator.availability.end)  # We take the maximum of the end of the availability of the operators
    return next_available_time"""
############################################################################################

# Initialization of the dataframe that will contain the assignments of the operators
operators_assignments: pd.DataFrame = pd.DataFrame(columns=list(Chronologically_Ordered_Steps.keys()), dtype=object)

time = simulation_start

modules_completed = sum(S___PDB_Shipment_of_modules_to_loading_sites.log["Entry_Date"].notna())

while (time < simulation_end and modules_completed < 50):

    generate_operators_availability(time)    # TODO: Duplication of computation, we could do it once for the day
    to_do: list = tasks_by_priority(time)
    was_a_task_assigned: bool = False
    if len(to_do) == 0:
        time += timedelta(hours=1)

    while len(to_do) > 0:

        # if two operators are available, and qualified for the task we assign them to the task
        task = to_do.pop(0)

        time = get_next_available_time_for_task(time, task)

        task_duration: Interval = Interval(time, time + task.required, task.name)

        operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

        if len(operators_available) < 2:
            print("Err")

        # For now we chose at random two of the available operators to perform the task, the law according to which we chose the operators might be reweighted according to their skills
        # Wire Bonding journée entière et 2 fois par semaine.

        chosen_operators: list = np.random.choice(operators_available, 2, replace=False)
        first_operator, second_operator = chosen_operators
        first_operator.availability.chop(time, time + task.required)
        second_operator.availability.chop(time, time + task.required)
        assigned_operators = (first_operator.name, second_operator.name)
        operators_assignments.loc[time, task.name] = assigned_operators

        update_log(task, time)
        time += task.required
        was_a_task_assigned = True

        print(f"task {task.name} assigned to operators {assigned_operators} at time {time}")
        break

    modules_completed = sum(S___PDB_Shipment_of_modules_to_loading_sites.log["Entry_Date"].notna())
        
"""        if not was_a_task_assigned :
            print("Error")
            time += timedelta(minutes=60)   # TODO: time = next available slot (which would be based on the next lab slot, the lab planning being the union of all plannings)"""
        

    



print("Computation finished here is the operators assignments : ")
print(operators_assignments)

C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Step.log = pd.concat([Step.log, df_Ready, df_WIP], ignore_index=True)   # TODO : maybe numpy arrays are better suited in this context, since it's a single type of datatype, in addition to that this line raises a warning
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Step.log = pd.concat([Step.log, df_Ready, df_WIP],

task Rec - PDB checks of flexes assigned to operators ('Operator3', 'Operator9') at time 2024-11-01 09:10:00
task Rec - Metrology and Visual inspection of flexes assigned to operators ('Operator8', 'Operator10') at time 2024-11-01 09:30:00
task Rec - Metrology weights of flexes assigned to operators ('Operator7', 'Operator6') at time 2024-11-01 09:32:00
task Rec - PDB checks of flexes assigned to operators ('Operator7', 'Operator5') at time 2024-11-01 09:42:00
task Rec - Metrology and Visual inspection of flexes assigned to operators ('Operator10', 'Operator3') at time 2024-11-01 10:02:00
task Rec - PDB checks of flexes assigned to operators ('Operator8', 'Operator3') at time 2024-11-01 10:12:00
task Rec - PDB checks of flexes assigned to operators ('Operator8', 'Operator10') at time 2024-11-01 10:22:00
task Rec - PDB checks of flexes assigned to operators ('Operator8', 'Operator9') at time 2024-11-01 10:32:00
task Rec - PDB checks of flexes assigned to operators ('Operator7', 'Operato

C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is de

task Rec - PDB checks of bare modules assigned to operators ('Operator5', 'Operator10') at time 2024-11-01 16:34:00
task Rec - Metrology and Visual inspection of bare modules assigned to operators ('Operator9', 'Operator10') at time 2024-11-01 16:54:00
task Rec - Metrology weights of bare modules assigned to operators ('Operator7', 'Operator10') at time 2024-11-01 16:56:00
task Rec - Sensor IV of bare modules assigned to operators ('Operator8', 'Operator10') at time 2024-11-01 17:16:00
task FA - Glue flex to bare module assigned to operators ('Operator9', 'Operator5') at time 2024-11-01 17:46:00
task Rec - Metrology weights of flexes assigned to operators ('Operator5', 'Operator1') at time 2024-11-01 17:48:00
task Rec - Metrology and Visual inspection of flexes assigned to operators ('Operator7', 'Operator9') at time 2024-11-04 09:20:00
task FA - visual inspection of module assigned to operators ('Operator8', 'Operator10') at time 2024-11-04 09:30:00
task FA - Metrology weights of modu

C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is de

...

task TC - Back side metrology assigned to operators ('Operator10', 'Operator9') at time 2024-11-29 09:31:00
task TC - PDB Upload of back side metrology assigned to operators ('Operator8', 'Operator1') at time 2024-11-29 09:32:00
task LTS - MHT-PFA assigned to operators ('Operator10', 'Operator3') at time 2024-11-29 10:32:00
task F - Warm assigned to operators ('Operator6', 'Operator10') at time 2024-11-29 10:42:00
task TC - PDB Upload of thermal cycling data assigned to operators ('Operator10', 'Operator3') at time 2024-11-29 10:43:00
task TC - Back side metrology assigned to operators ('Operator3', 'Operator9') at time 2024-11-29 11:03:00
task TC - PDB Upload of back side metrology assigned to operators ('Operator3', 'Operator8') at time 2024-11-29 11:04:00
task LTS - MHT-PFA assigned to operators ('Operator1', 'Operator9') at time 2024-11-29 14:00:00
task F - Cold assigned to operators ('Operator10', 'Operator9') at time 2024-11-29 14:10:00
task F - Warm assigned to operators ('Opera

C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)


task FA - visual inspection of module assigned to operators ('Operator10', 'Operator7') at time 2024-11-29 15:30:00
task FA - Metrology weights of module assigned to operators ('Operator10', 'Operator5') at time 2024-11-29 15:32:00
task P - visual inspection after transit assigned to operators ('Operator3', 'Operator8') at time 2024-11-29 15:42:00
task P - Demasking assigned to operators ('Operator1', 'Operator7') at time 2024-11-29 16:02:00
task P - visual inspection after unmasking assigned to operators ('Operator5', 'Operator8') at time 2024-11-29 16:12:00
task P - Post Parylene Warm assigned to operators ('Operator8', 'Operator5') at time 2024-11-29 16:22:00
task WB - Prepare wire-bonding requests assigned to operators ('Operator10', 'Operator4') at time 2024-11-29 16:42:00
task WB - Wire-bonding assigned to operators ('Operator6', 'Operator2') at time 2024-12-02 11:00:00
task F - X-ray assigned to operators ('Operator8', 'Operator9') at time 2024-12-02 11:20:00
task S - Packaging 

C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  task.log = pd.concat([task.log, df], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_23764\830001035.py:263: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is de

task WBP - PDB Upload of visual inspection mass and resistance assigned to operators ('Operator10', 'Operator7') at time 2024-12-02 13:23:00
task WBP - MHT-PFA assigned to operators ('Operator6', 'Operator3') at time 2024-12-02 14:23:00
task TC - Cycle assigned to operators ('Operator6', 'Operator10') at time 2024-12-02 14:43:00
task P - Post Parylene Cold assigned to operators ('Operator6', 'Operator9') at time 2024-12-02 14:53:00
task P - Post Parylene cold chiller cooldown assigned to operators ('Operator8', 'Operator3') at time 2024-12-02 15:03:00
task I - Cold Electrical Testing assigned to operators ('Operator3', 'Operator8') at time 2024-12-02 15:13:00
task WB - PDB Upload of wire-bonding pull-test results assigned to operators ('Operator2', 'Operator7') at time 2024-12-02 15:23:00
task I - Warm Electrical Testing assigned to operators ('Operator9', 'Operator5') at time 2024-12-02 15:33:00
task P - visual inspection after transit assigned to operators ('Operator1', 'Operator6') 

In [2]:
S___PDB_Shipment_of_modules_to_loading_sites.log

,Entry_Date,Exit_Date
0,2024-12-02 11:25:00,NaT
1,2024-12-02 16:28:00,NaT
2,2024-12-04 09:25:00,NaT
3,2024-12-06 09:35:00,NaT
4,2024-12-06 10:20:00,NaT
5,2024-12-06 15:27:00,NaT
6,2024-12-11 09:45:00,NaT
7,2024-12-11 14:47:00,NaT
8,2024-12-11 15:32:00,NaT
9,2024-12-16 09:35:00,NaT


In [6]:
sum(S___PDB_Shipment_of_modules_to_loading_sites.log["Entry_Date"].notna())

50

In [ ]:
import pandas as pd
import numpy as np
import json
from dateutil import rrule, parser
from dataclasses import dataclass, field
from datetime import date, datetime, timedelta
from intervaltree import Interval, IntervalTree

# Definition of the different classes
@dataclass
class Operator:
    name: str
    skills: list
    holidays: IntervalTree
    availability: IntervalTree = field(default_factory=IntervalTree)
    shift: list = field(default_factory=list)   # Is it used ?

@dataclass
class Step:
    name: str
    previous_steps: list
    duration: timedelta
    required: timedelta
    capacity: int
    log: pd.DataFrame

    def __repr__(self):
        return f"Step {self.name}, duration {self.duration}, capacity {self.capacity}, log \n{self.log}"

# Load data from the JSON configuration file
with open("SimulatorInputs.json", "r") as file:
    data = json.load(file)

# Extract the simulation parameters
json_simulation_start: str = data['SimulationParameters']['simulation_start']
simulation_start: datetime = datetime.fromisoformat(json_simulation_start)

json_simeulation_end: str = data['SimulationParameters']['simulation_end']
simulation_end: datetime = datetime.fromisoformat(json_simeulation_end)

#modules_planned_deliveries: str = data["ComponentArrivalTimes"]["bare modules"]

# Initialize instances of the operator class
# All the operators are stored in the operators list
operators = []

# Initialize the CERN holidays to the good format
CERN_holidays_series = pd.Series(data["CERNHolidays"])
CERN_holidays_series[:] = CERN_holidays_series.apply(lambda x: [datetime.fromisoformat(date) for date in x])

for id, operator in enumerate(data["Operators"]):
    # Loads the operator's individual holidays
    individual_holidays = pd.Series(data["OperatorHolidays"][f"Operator{id + 1}"])
    individual_holidays[:] = individual_holidays.apply(lambda x: [datetime.fromisoformat(date) for date in x])

    # Merges the individual holidays with the CERN holidays
    operator_holidays = pd.concat([individual_holidays, CERN_holidays_series])
    operator_holidays[:] = operator_holidays.apply(lambda x: Interval(x[0], x[1], "holiday"))

    # Converts the pandas series to a list and then to an interval tree
    operator_holidays_list = operator_holidays.tolist()
    operator_holidays_tree = IntervalTree(operator_holidays_list)

    globals()[f"Operator{id + 1}"] = Operator(name=f"Operator{id + 1}", skills=data["Operators"][operator], holidays=operator_holidays_tree)
    operators.append(globals()[f"Operator{id + 1}"])


# Initialization of the Step class instances, they are stored in the "Chronologically_Ordered_Steps" dict, the member log is a dataframe that will contain
# the entry and exit dates of the modules at each step

Chronologically_Ordered_Steps = {}
for step in data["StagesAndSteps"]:
    step_name = step["Step"].replace("-", "_").replace(" ", "_")    # Just converting the name to python valid name without blank spaces and


    if step["Previous"][0] == "None":
        globals()[step_name] = Step(name=step["Step"], previous_steps=[None], duration=timedelta(minutes = step["Duration"]),required=timedelta(minutes = step["Required"]), capacity= step["Capacity"],log=pd.DataFrame(columns=["Entry_Date", "Exit_Date"]))

    else:
        previous_steps = []
        for prev_step in step["Previous"]:
            previous_steps.append(Chronologically_Ordered_Steps[prev_step])
            globals()[step_name] = Step(name=step["Step"], previous_steps=previous_steps, duration=timedelta(minutes = step["Duration"]), required=timedelta(minutes = step["Required"]), capacity= step["Capacity"],log=pd.DataFrame(columns=["Entry_Date", "Exit_Date"]))
    Chronologically_Ordered_Steps |= {step["Step"]: globals()[step_name]}


In [ ]:
# Extract the holidays, and daily shift from the data
dict_int_to_day = {"0": "Monday", "1": "Tuesday", "2": "Wednesday", "3": "Thursday", "4": "Friday"}

def generate_operators_availability(date: datetime) -> None:
# Let's try to generate the availability of the operator day wise
    weekday: str = str(date.weekday())

    for operator in operators:
    # Reset the availability of the operator first
        operator.availability = IntervalTree()

        if len(operator.holidays[date]) or date.weekday() in {5,6}:  # Check if the day is either in the holidays or weekend
            continue

        else:

            today_shift = data["OperatorWorkHours"][operator.name][dict_int_to_day[weekday]]
            list_today_shift = list(today_shift.keys())

            # TODO: check for even number of time constraints
            beginning_slot = list_today_shift[::2]  # Extract the beginning and ending time of the different operator's daily slots
            ending_slot = list_today_shift[1::2]    # Implicitly we assume that there is an even number of time constraints (namely the beginning and the ending of the time slot)

            for begin, end in zip(beginning_slot, ending_slot):
                start_time = str(date)[:10] + " " + today_shift[begin]
                start_time = datetime.fromisoformat(start_time)
                end_time = str(date)[:10] + " " + today_shift[end]
                end_time = datetime.fromisoformat(end_time)
                operator.availability.add(Interval(start_time, end_time, "idle"))



In [ ]:
def generate_lab_hours(date: datetime) -> IntervalTree:
    lab_hours: IntervalTree = IntervalTree()
    generate_operators_availability(date)
    for operator in operators:
        lab_hours |= operator.availability
    return lab_hours

In [ ]:
date = datetime.now()
generate_lab_hours(date)

In [ ]:
def is_task_assignable(tree: IntervalTree, task_duration: Interval) -> bool:    #TODO: Manually implement IntervalTree.py and add this as a method of IntervalTree.
    assignable: bool = False
    for interval in tree:
        if interval.begin <= task_duration.begin and task_duration.end <= interval.end:
            assignable = True
            break
    return assignable

is_task_assignable(Operator1.availability, task_duration)


In [ ]:
def get_next_available_time_for_task(time: datetime, task: Step) -> datetime:
    task_duration: Interval = Interval(time, time + task.required, task.name)
    operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

    new_time: datetime = time
    while len(operators_available) < 2:
        lab_hours = generate_lab_hours(new_time)
        has_moved: bool = False
        for interval in sorted(lab_hours):
            if interval.begin > time:
                time = interval.begin
                has_moved = True
                break

        if has_moved == False:
            new_time += timedelta(days=1)
        else:
            task_duration: Interval = Interval(time, time + task.required, task.name)
            operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

    return time


In [ ]:
time = datetime.now() + timedelta(days=1, hours=3)
print(time, "\n")
generate_operators_availability(time)

task = FA___Glue_flex_to_bare_module

task_duration = Interval(time, time + task.required, task.name)

operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

print(get_next_available_time_for_task(time, task), "\n")

In [ ]:
time = datetime.now() + timedelta(hours=3)
generate_operators_availability(time)

task = FA___Glue_flex_to_bare_module

task_duration = Interval(time, time + task.required, task.name)

operators_available: list = [operator for operator in operators if is_task_assignable(operator.availability, task_duration) and task.name in operator.skills]

get_next_available_time_for_task(time, task)